In [4]:
ls

Cleaning Data - US President & Vice President.ipynb
General2016.csv
README.md
__pycache__/
config.py
proposal.txt
replies.py


In [5]:
import pandas as pd
import tweepy
import time
import json
from replies import *
from pprint import pprint
from config import consumer_key, consumer_secret, access_token, access_token_secret

In [6]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [24]:
#import csv
file = "2016ElectionDataByState.csv"
df = pd.read_csv(file)
del df['Unnamed: 3']
df


,State,Clinton,Trump
0,Alabama,"729,547","1,318,255"
1,Alaska,"116,454","163,387"
2,Arizona,"1,161,167","1,252,401"
3,Arkansas,"380,494","684,872"
4,California,"8,753,792","4,483,814"
5,Colorado,"1,338,870","1,202,484"
6,Connecticut,"897,572","673,215"
7,Delaware,"235,603","185,127"
8,D. C.,"282,830","12,723"
9,Florida,"4,504,975","4,617,886"


In [8]:
# #show US Pres & Vice Pres only
# clean_df = df.loc[(df["Race"] == 'US President & Vice President')]
# clean_df.head()

,Precinct,Race,LEG,CC,CG,CounterGroup,Party,CounterType,SumOfCount
218,ADAIR,US President & Vice President,45.0,3.0,1,Total,CPN,Darrell L. Castle & Scott N. Bradley,0
219,ADAIR,US President & Vice President,45.0,3.0,1,Total,DPN,Hillary Clinton & Tim Kaine,266
220,ADAIR,US President & Vice President,45.0,3.0,1,Total,GPN,Jill Stein & Ajamu Baraka,1
221,ADAIR,US President & Vice President,45.0,3.0,1,Total,LPN,Gary Johnson & Bill Weld,14
222,ADAIR,US President & Vice President,45.0,3.0,1,Total,NP,Registered Voters,519


In [9]:
# df = clean_df.groupby(["CounterType","Precinct","Party"])["SumOfCount"].sum().reset_index()  
# df.head()

,CounterType,Precinct,Party,SumOfCount
0,Alyson Kennedy & Osborne Hart,ADAIR,SWN,0
1,Alyson Kennedy & Osborne Hart,ALDARRA,SWN,0
2,Alyson Kennedy & Osborne Hart,ALDER SPRINGS,SWN,1
3,Alyson Kennedy & Osborne Hart,ALDERWOOD,SWN,0
4,Alyson Kennedy & Osborne Hart,ALG 30-0013,SWN,4


In [25]:
# df.loc[(df["Party"].isin(['RPN','DPN']))].reset_index(drop=True)  

In [26]:
# Target Term
target_user = "@realDonaldTrump"

In [35]:
# Get all tweets from target user
tweet_list = []
public_tweets = api.user_timeline(target_user, page=1)
original_tweet = 0
# Loop through all tweets
for i, tweet in enumerate(public_tweets):
    tweet_list.append(tweet)
    replies_list = []
    orginal_tweet = tweet.get('id')

    replies = api.search(q='to:@realDonaldTrump',since_id=tweet.get('id'),count=100,geocode='39.213780,-99.504832,2000mi')
    while len(replies.get('statuses')) > 0 and replies.get('statuses')[0].get('id') > original_tweet:
        for reply in replies["statuses"]:
            if reply.get('coordinates') != None:
                replies_list.append(reply) 
        replies = api.search(q='to:@realDonaldTrump',since_id=reply.get('id'), count=100,geocode='39.213780,-99.504832,2000mi')
        original_tweet = reply.get('id')  
        if (len(replies_list) > 4000):
            break
    break

    

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [34]:
len(tweet_list), len(replies_list)

(1, 0)

In [14]:
pprint(tweet_list[0].get('id')), pprint(tweet_list[0].get('created_at')), pprint(tweet_list[0].get('text'))

1004478326336040960
'Wed Jun 06 21:41:09 +0000 2018'
('Thank you to everyone at @FEMA HQ for today’s briefing on preparations for '
 'the upcoming hurricane season. Disaster… https://t.co/SszyuwMdWp')


(None, None, None)

In [57]:
for reply in replies_list:
    pprint(reply.get('id'))
    pprint(reply.get('created_at'))
    pprint(reply.get('text'))

1004492477607407616
'Wed Jun 06 22:37:23 +0000 2018'
'@realDonaldTrump @fema Two words:  Puerto Rico.'
1004492477217193984
'Wed Jun 06 22:37:23 +0000 2018'
('RT @trumpism_45: @realDonaldTrump The 2 million bikers are headed to '
 'Washington D.C. to confront The Deep State Mueller team to demand the…')
1004492473836752897
'Wed Jun 06 22:37:22 +0000 2018'
('@realDonaldTrump Mr. President once again you have forgotten that the '
 'Republican control both houses of Congress.')
1004492470439247872
'Wed Jun 06 22:37:22 +0000 2018'
'@realDonaldTrump @fema So what federal support are you offering #PuertoRico?'
1004492466815262721
'Wed Jun 06 22:37:21 +0000 2018'
('RT @DianeQuinnUS: @realDonaldTrump Pardoning yourself as President only '
 'leads to corruption with power left unchecked.  This is the exact o…')
1004492466400178176
'Wed Jun 06 22:37:21 +0000 2018'
'@realDonaldTrump the big gamble'
1004492465741746177
'Wed Jun 06 22:37:21 +0000 2018'
'@realDonaldTrump  https://t.co/3VyERc2YC4'
100

In [15]:
# pprint(replies_list[0].get('id')), pprint(replies_list[0].get('created_at')), pprint(replies_list[0].get('text'))

In [49]:
# pprint(replies_list[1])